## 1472. Design Browser History
- Description:
  <blockquote>
    You have a **browser** of one tab where you start on the `homepage` and you can visit another `url`, get back in the history number of `steps` or move forward in the history number of `steps`.
   
  Implement the `BrowserHistory` class:
   
  - `BrowserHistory(string homepage)` Initializes the object with the `homepage` of the browser.
  - `void visit(string url)` Visits `url` from the current page. It clears up all the forward history.
  - `string back(int steps)` Move `steps` back in history. If you can only return `x` steps in the history and `steps > x`, you will return only `x` steps. Return the current `url` after moving back in history **at most** `steps`.
  - `string forward(int steps)` Move `steps` forward in history. If you can only forward `x` steps in the history and `steps > x`, you will forward only `x` steps. Return the current `url` after forwarding in history **at most** `steps`.
   
  **Example:**
  Input:["BrowserHistory","visit","visit","visit","back","back","forward","visit","forward","back","back"]
  `["leetcode.com"], ["google.com"],["facebook.com"],["youtube.com"],[1],[1],[1],["linkedin.com"],[2],[2],[7]`Output:[null,null,null,null,"facebook.com","google.com","facebook.com",null,"linkedin.com","google.com","leetcode.com"]Explanation:BrowserHistory browserHistory = new BrowserHistory("leetcode.com");
  browserHistory.visit("google.com");       // You are in "leetcode.com". Visit "google.com"
  browserHistory.visit("facebook.com");     // You are in "google.com". Visit "facebook.com"
  browserHistory.visit("youtube.com");      // You are in "facebook.com". Visit "youtube.com"
  browserHistory.back(1);                   // You are in "youtube.com", move back to "facebook.com" return "facebook.com"
  browserHistory.back(1);                   // You are in "facebook.com", move back to "google.com" return "google.com"
  browserHistory.forward(1);                // You are in "google.com", move forward to "facebook.com" return "facebook.com"
  browserHistory.visit("linkedin.com");     // You are in "facebook.com". Visit "linkedin.com"
  browserHistory.forward(2);                // You are in "linkedin.com", you cannot move forward any steps.
  browserHistory.back(2);                   // You are in "linkedin.com", move back two steps to "facebook.com" then to "google.com". return "google.com"
  browserHistory.back(7);                   // You are in "google.com", you can move back only one step to "leetcode.com". return "leetcode.com"
   
  **Constraints:**
   
  - `1 <= homepage.length <= 20`
  - `1 <= url.length <= 20`
  - `1 <= steps <= 100`
  - `homepage` and `url` consist of  '.' or lower case English letters.
  - At most `5000` calls will be made to `visit`, `back`, and `forward`.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/design-browser-history/description)

- Topics: Stacks, Doubly Linked List, Arrays

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, Most Efficient, Dynamic Array Pointer Tracking Approach
Key Advantage:
Unlike the stack-based approaches (which required O(steps) time for back/forward), this solution uses index arithmetic to move the pointer in constant time. This is more efficient for large steps values.

Solution description
- Time Complexity: O(1)
  - visit: O(1) (amortized for list append/overwriting).
  - back/forward: O(1) (direct index calculation, no loops).
- Space Complexity: O(N)
  - where N is the number of visited URLs (stores all URLs in a single list).

In [ ]:
class BrowserHistory:
    def __init__(self, homepage: str):
        # 'homepage' is the first visited URL.
        self.visited_URLs = [homepage]
        self.curr_URL, self.last_URL = 0, 0

    def visit(self, url: str) -> None:
        self.curr_URL += 1
        if len(self.visited_URLs) > self.curr_URL:
            # We have enough space in our array to overwrite an old 'url' entry with new one.
            self.visited_URLs[self.curr_URL] = url
        else:
            # We have to insert a new 'url' entry at the end.
            self.visited_URLs.append(url)
        # This 'url' will be last URL if we try to go forward.
        self.last_URL = self.curr_URL

    def back(self, steps: int) -> str:
        # Move 'curr_URL' pointer in left direction.
        self.curr_URL = max(0, self.curr_URL - steps)
        return self.visited_URLs[self.curr_URL]

    def forward(self, steps: int) -> str:
        # Move 'curr_URL' pointer in right direction.
        self.curr_URL = min(self.last_URL, self.curr_URL + steps)
        return self.visited_URLs[self.curr_URL]

### Solution 1.1, Dynamic Array Pointer with additional requirements of haveVisited() method and multiple tabs
Solution description
- Time Complexity: O(1)
- Space Complexity: O(N)

In [ ]:
class Tab:
    def __init__(self, homepage: str):
        # Dynamic array to store history
        self.history = [homepage]
        # Current index pointer
        self.curr = 0
        # The boundary of how far 'forward' we can go
        self.last = 0

    def visit(self, url: str):
        self.curr += 1
        # If we are inserting a new URL in the middle of history, 
        # we overwrite the next slot.
        if self.curr < len(self.history):
            self.history[self.curr] = url
        else:
            self.history.append(url)
        # Any previous 'forward' history is now invalidated
        self.last = self.curr

    def back(self, steps: int) -> str:
        self.curr = max(0, self.curr - steps)
        return self.history[self.curr]

    def forward(self, steps: int) -> str:
        self.curr = min(self.last, self.curr + steps)
        return self.history[self.curr]


class MultiTabBrowser:
    def __init__(self, homepage: str):
        # initializing a set that contains one element: the homepage
        self.global_visited = {homepage}
        # Start with one default tab (tab_id 0)
        self.tabs = {0: Tab(homepage)}
        self.active_tab_id = 0

    def open_new_tab(self, tab_id: int, homepage: str):
        if tab_id not in self.tabs:
            self.tabs[tab_id] = Tab(homepage)
            self.global_visited.add(homepage)

    def switch_tab(self, tab_id: int):
        if tab_id in self.tabs:
            self.active_tab_id = tab_id

    def visit(self, url: str):
        # Visit URL on the currently active tab
        current_tab = self.tabs[self.active_tab_id]
        current_tab.visit(url)
        self.global_visited.add(url)

    def back(self, steps: int) -> str:
        current_tab = self.tabs[self.active_tab_id]
        return current_tab.back(steps)

    def forward(self, steps: int) -> str:
        current_tab = self.tabs[self.active_tab_id]
        return current_tab.forward(steps)

    def haveVisited(self, url: str) -> bool:
        # O(1) check across all tabs
        return url in self.global_visited

def run_tests():
    # 1. Initialize Browser with Tab 0
    browser = MultiTabBrowser("google.com")
    print(f"Initial: {browser.haveVisited('google.com')}") # Expected: True
    
    # 2. Basic Navigation on Tab 0
    browser.visit("leetcode.com")
    browser.visit("github.com")
    print(f"Back 1: {browser.back(1)}")       # Expected: leetcode.com
    print(f"Forward 1: {browser.forward(1)}") # Expected: github.com
    
    # 3. Multiple Tabs: Open Tab 1
    browser.open_new_tab(1, "openai.com")
    browser.switch_tab(1)
    browser.visit("youtube.com")
    
    # 4. Check Global 'haveVisited'
    # Tab 1 has never been to github, but Tab 0 has.
    print(f"Visited GitHub? {browser.haveVisited('github.com')}") # Expected: True
    print(f"Visited YouTube? {browser.haveVisited('youtube.com')}") # Expected: True
    print(f"Visited Reddit? {browser.haveVisited('reddit.com')}")   # Expected: False

    # 5. Independent Navigation Check
    # Switch back to Tab 0. Its pointer should still be at github.com
    browser.switch_tab(0)
    print(f"Tab 0 current (should be github): {browser.back(0)}") # Expected: github.com
    
    # 6. Overwriting History (The "Branching" Test)
    # On Tab 0: google -> leetcode -> github. 
    # Current is github. Go back to leetcode, then visit stackoverflow.
    # forward(1) should no longer lead to github.
    browser.back(1) # Now at leetcode
    browser.visit("stackoverflow.com") 
    print(f"Tab 0 Forward after new visit: {browser.forward(1)}") # Expected: stackoverflow.com (no github)
    
    print("All tests completed!")

run_tests()

### Solution 2, Most Intuitive, Doubly Linked List

Time Complexity:

    visit: O(1) (creating a new node and updating pointers).
    back/forward: O(steps) (each step moves the pointer once, with O(1) per move).

Space Complexity:

    O(n), where n is the number of visited URLs.
    Explanation: Each URL corresponds to a node in the linked list, so space scales linearly with the number of visits.

Why this is efficient:
The linked list avoids list slicing or resizing (unlike array-based solutions), making pointer movements purely O(1) per step. This ensures optimal performance for back/forward operations when steps is small.

Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
class Node:
    def __init__(self, url: str):
        self.data = url
        self.prev, self.next = None, None

class BrowserHistory:
    def __init__(self, homepage: str):
        # 'homepage' is the first visited URL.
        self.linked_list_head = Node(homepage)
        self.current = self.linked_list_head

    def visit(self, url: str) -> None:
        # Insert new node 'url' in the right of current node.
        new_node = Node(url)
        self.current.next = new_node
        new_node.prev = self.current
        # Make this new node as current node now.
        self.current = new_node

    def back(self, steps: int) -> str:
        # Move 'current' pointer in left direction.
        while steps and self.current.prev:
            self.current = self.current.prev
            steps -= 1
        return self.current.data

    def forward(self, steps: int) -> str:
        # Move 'current' pointer in right direction.
        while steps and self.current.next:
            self.current = self.current.next
            steps -= 1
        return self.current.data

### Solution 3, Most Basic, Two Stacks

Time Complexity:

    visit: O(1) (appending to history and resetting future are constant-time operations).
    back/forward: O(steps) (each step requires a single pop/append operation on lists, which are O(1) per operation).

Space Complexity:

    O(n), where n is the total number of visited URLs.
    Explanation: The history and future lists store all URLs visited (excluding the current URL), so their combined size grows linearly with the number of visits.

Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
class BrowserHistory:

    def __init__(self, homepage: str):
        self.current = homepage
        self.history = []
        self.future = []


    def visit(self, url: str) -> None:
        self.history.append(self.current)
        self.current = url
        self.future = []
        

    def back(self, steps: int) -> str:
        while steps and self.history:
            self.future.append(self.current)
            self.current = self.history.pop()
            steps -= 1
        
        return self.current
        

    def forward(self, steps: int) -> str:
        while steps and self.future:
            self.history.append(self.current)
            self.current = self.future.pop()
            steps -= 1
        
        return self.current
        


# Your BrowserHistory object will be instantiated and called as such:
# obj = BrowserHistory(homepage)
# obj.visit(url)
# param_2 = obj.back(steps)
# param_3 = obj.future(steps)